In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [16]:
prec = pd.read_csv('data/MH25_vaisalawxt520prec_2017.csv')
wind = pd.read_csv('data/MH25_vaisalawxt520windpth_2017.csv')
temp = pd.read_csv('data/MH30_temperature_rock_2017.csv')
radio = pd.read_csv('data/MH15_radiometer__conv_2017.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 16 columns):
time                                         8760 non-null object
position []                                  8760 non-null float64
irradiance_shortwave_up [Wm^-2]              8760 non-null float64
irradiance_shortwave_down [Wm^-2]            8760 non-null float64
albedo []                                    8760 non-null float64
net_solar_radiation [Wm^-2]                  8760 non-null float64
irradiance_longwave_up_notemp [Wm^-2]        8760 non-null float64
irradiance_longwave_down_notemp [Wm^-2]      8760 non-null float64
net_far_infrared_radiation_notemp [Wm^-2]    8760 non-null float64
temperature [°C]                             8760 non-null float64
irradiance_longwave_up [Wm^-2]               8760 non-null float64
irradiance_longwave_down [Wm^-2]             8760 non-null float64
net_far_infrared_radiation [Wm^-2]           8760 non-null float64
sky_temperature [°C]   

In [9]:
prec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 10 columns):
time                                     8760 non-null object
position []                              8749 non-null float64
rain_accumulation [mm]                   8760 non-null float64
rain_duration [s]                        8760 non-null float64
rain_intensity [mm/h]                    8526 non-null float64
rain_peak_intensity [mm/h]               8526 non-null float64
hail_accumulation [hits/cm^2]            8760 non-null float64
hail_duration [s]                        8760 non-null float64
hail_intensity [hits cm^-2 h^-1]         8526 non-null float64
hail_peak_intensity [hits cm^-2 h^-1]    8526 non-null float64
dtypes: float64(9), object(1)
memory usage: 684.5+ KB


In [18]:
temp_inner = pd.merge(left= temp, right=prec, left_on='time',right_on='time')
temp_inner = pd.merge(left= temp_inner, right=wind, left_on='time',right_on='time')
temp_inner = pd.merge(left=temp_inner, right=radio, left_on='time',right_on='time')

In [20]:
temp_inner.head()

,time,position []_x,temperature_5cm [°C],temperature_10cm [°C],temperature_20cm [°C],temperature_30cm [°C],temperature_50cm [°C],temperature_100cm [°C],position []_y,rain_accumulation [mm],...,irradiance_longwave_up_notemp [Wm^-2],irradiance_longwave_down_notemp [Wm^-2],net_far_infrared_radiation_notemp [Wm^-2],temperature [°C],irradiance_longwave_up [Wm^-2],irradiance_longwave_down [Wm^-2],net_far_infrared_radiation [Wm^-2],sky_temperature [°C],ground_temperature [°C],net_radiation [Wm^-2]
0,2017-01-01T00:00:00Z,30,-9.4934,-9.1609,-8.7378,-8.4231,-7.9328,-7.1922,25.0,0.0,...,-105.7430,-11.7359,-94.0071,-8.4971,172.4138,266.4209,-94.0071,-38.3234,-11.3340,-103.3271
1,2017-01-01T01:00:00Z,30,-9.5680,-9.2330,-8.8027,-8.4813,-7.9573,-7.2040,25.0,0.0,...,-103.8140,-10.4743,-93.3397,-9.1421,171.6424,264.9821,-93.3397,-38.5866,-11.6882,-103.3083
2,2017-01-01T02:00:00Z,30,-9.6568,-9.3084,-8.8668,-8.5235,-7.9845,-7.2081,25.0,0.0,...,-102.9833,-10.3014,-92.6819,-9.3633,171.5504,264.2323,-92.6819,-38.6181,-11.8733,-102.9365
3,2017-01-01T03:00:00Z,30,-9.7203,-9.3797,-8.9384,-8.5756,-8.0103,-7.2166,25.0,0.0,...,-102.3275,-10.3100,-92.0176,-9.4608,171.8012,263.8187,-92.0176,-38.5324,-11.9756,-101.0145
4,2017-01-01T04:00:00Z,30,-9.7547,-9.4270,-8.9953,-8.6287,-8.0473,-7.2120,25.0,0.0,...,-104.1133,-11.5732,-92.5401,-9.1177,171.4444,263.9845,-92.5401,-38.6543,-11.9346,-101.7435


In [21]:
temp_inner.columns

Index(['time', 'position []_x', 'temperature_5cm [°C]',
       'temperature_10cm [°C]', 'temperature_20cm [°C]',
       'temperature_30cm [°C]', 'temperature_50cm [°C]',
       'temperature_100cm [°C]', 'position []_y', 'rain_accumulation [mm]',
       'rain_duration [s]', 'rain_intensity [mm/h]',
       'rain_peak_intensity [mm/h]', 'hail_accumulation [hits/cm^2]',
       'hail_duration [s]', 'hail_intensity [hits cm^-2 h^-1]',
       'hail_peak_intensity [hits cm^-2 h^-1]', 'position []_x',
       'wind_direction_minimum [°]', 'wind_direction_average [°]',
       'wind_direction_maximum [°]', 'wind_speed_minimum [km/h]',
       'wind_speed_average [km/h]', 'wind_speed_maximum [km/h]',
       'temp_air [°C]', 'temp_internal [°C]', 'relative_humidity [%RH]',
       'air_pressure [hPa]', 'position []_y',
       'irradiance_shortwave_up [Wm^-2]', 'irradiance_shortwave_down [Wm^-2]',
       'albedo []', 'net_solar_radiation [Wm^-2]',
       'irradiance_longwave_up_notemp [Wm^-2]',
       

In [49]:
diff = temp_inner['temperature_100cm [°C]'] - temp_inner['temperature_5cm [°C]']

In [137]:
del temp_inner['time']

KeyError: 'time'

In [151]:
del temp_inner['position []_x']

In [153]:
diff.shape

(8760,)

In [154]:
del temp_inner['delta_t']
temp_inner.insert(0,'delta_t',diff)

In [155]:
abs(temp_inner.corr()['delta_t']).sort_values(ascending=False)

delta_t                                      1.000000
temperature [°C]                             0.758564
ground_temperature [°C]                      0.754324
irradiance_longwave_down [Wm^-2]             0.740201
temp_air [°C]                                0.726590
temperature_5cm [°C]                         0.720564
temperature_10cm [°C]                        0.690005
temp_internal [°C]                           0.654179
temperature_20cm [°C]                        0.634312
air_pressure [hPa]                           0.614342
temperature_30cm [°C]                        0.581249
temperature_50cm [°C]                        0.500326
sky_temperature [°C]                         0.486733
irradiance_longwave_up [Wm^-2]               0.473623
temperature_100cm [°C]                       0.393467
irradiance_longwave_down_notemp [Wm^-2]      0.344815
net_solar_radiation [Wm^-2]                  0.300936
net_radiation [Wm^-2]                        0.293974
wind_direction_average [°]  

In [265]:
temp_inner.reset_index()

,index,delta_t,temperature_5cm _°C_,temperature_10cm _°C_,temperature_20cm _°C_,temperature_30cm _°C_,temperature_50cm _°C_,temperature_100cm _°C_,rain_accumulation _mm_,rain_duration _s_,...,irradiance_longwave_up_notemp _Wm^-2_,irradiance_longwave_down_notemp _Wm^-2_,net_far_infrared_radiation_notemp _Wm^-2_,temperature _°C_,irradiance_longwave_up _Wm^-2_,irradiance_longwave_down _Wm^-2_,net_far_infrared_radiation _Wm^-2_,sky_temperature _°C_,ground_temperature _°C_,net_radiation _Wm^-2_
0,0,2.3012,-9.4934,-9.1609,-8.7378,-8.4231,-7.9328,-7.1922,0.00,0.0,...,-105.7430,-11.7359,-94.0071,-8.4971,172.4138,266.4209,-94.0071,-38.3234,-11.3340,-103.3271
1,1,2.3640,-9.5680,-9.2330,-8.8027,-8.4813,-7.9573,-7.2040,0.00,0.0,...,-103.8140,-10.4743,-93.3397,-9.1421,171.6424,264.9821,-93.3397,-38.5866,-11.6882,-103.3083
2,2,2.4487,-9.6568,-9.3084,-8.8668,-8.5235,-7.9845,-7.2081,0.00,0.0,...,-102.9833,-10.3014,-92.6819,-9.3633,171.5504,264.2323,-92.6819,-38.6181,-11.8733,-102.9365
3,3,2.5037,-9.7203,-9.3797,-8.9384,-8.5756,-8.0103,-7.2166,0.00,0.0,...,-102.3275,-10.3100,-92.0176,-9.4608,171.8012,263.8187,-92.0176,-38.5324,-11.9756,-101.0145
4,4,2.5427,-9.7547,-9.4270,-8.9953,-8.6287,-8.0473,-7.2120,0.00,0.0,...,-104.1133,-11.5732,-92.5401,-9.1177,171.4444,263.9845,-92.5401,-38.6543,-11.9346,-101.7435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,8755,-3.5026,-7.6313,-7.7113,-8.0290,-8.6597,-9.8506,-11.1339,3336.84,614730.0,...,-99.5915,-11.8236,-87.7679,-5.1304,192.9931,280.7611,-87.7679,-31.6097,-7.8799,-94.8729
8756,8756,-3.2265,-7.8590,-7.9045,-8.1471,-8.6852,-9.7932,-11.0855,3336.84,614730.0,...,-94.9037,-10.4646,-84.4391,-5.9257,194.2252,278.6643,-84.4391,-31.2295,-8.3767,-90.9528
8757,8757,-3.0027,-8.0373,-8.0760,-8.2650,-8.7243,-9.7447,-11.0400,3336.84,614730.0,...,-89.8945,-9.8255,-80.0690,-6.2716,197.7397,277.8087,-80.0690,-30.2203,-8.5808,-85.6891
8758,8758,-2.7023,-8.2845,-8.2529,-8.3710,-8.7687,-9.7071,-10.9868,3229.20,594900.0,...,-91.2361,-9.9526,-81.2835,-7.1116,192.7957,274.0791,-81.2835,-31.6742,-9.4730,-88.2355


In [266]:
n = temp_inner.shape[0]
train_ind = int(np.round(0.7 * n))
print(train_ind)

6132


In [267]:
temp_inner.columns = temp_inner.columns.str.replace("[", "_")
temp_inner.columns = temp_inner.columns.str.replace("]", "_")

In [276]:
temp_inner = temp_inner.fillna(0)
temp_inner.replace([np.inf, -np.inf], 0)

,delta_t,temperature_5cm _°C_,temperature_10cm _°C_,temperature_20cm _°C_,temperature_30cm _°C_,temperature_50cm _°C_,temperature_100cm _°C_,rain_accumulation _mm_,rain_duration _s_,rain_intensity _mm/h_,...,irradiance_longwave_up_notemp _Wm^-2_,irradiance_longwave_down_notemp _Wm^-2_,net_far_infrared_radiation_notemp _Wm^-2_,temperature _°C_,irradiance_longwave_up _Wm^-2_,irradiance_longwave_down _Wm^-2_,net_far_infrared_radiation _Wm^-2_,sky_temperature _°C_,ground_temperature _°C_,net_radiation _Wm^-2_
0,2.3012,-9.4934,-9.1609,-8.7378,-8.4231,-7.9328,-7.1922,0.00,0.0,0.0,...,-105.7430,-11.7359,-94.0071,-8.4971,172.4138,266.4209,-94.0071,-38.3234,-11.3340,-103.3271
1,2.3640,-9.5680,-9.2330,-8.8027,-8.4813,-7.9573,-7.2040,0.00,0.0,0.0,...,-103.8140,-10.4743,-93.3397,-9.1421,171.6424,264.9821,-93.3397,-38.5866,-11.6882,-103.3083
2,2.4487,-9.6568,-9.3084,-8.8668,-8.5235,-7.9845,-7.2081,0.00,0.0,0.0,...,-102.9833,-10.3014,-92.6819,-9.3633,171.5504,264.2323,-92.6819,-38.6181,-11.8733,-102.9365
3,2.5037,-9.7203,-9.3797,-8.9384,-8.5756,-8.0103,-7.2166,0.00,0.0,0.0,...,-102.3275,-10.3100,-92.0176,-9.4608,171.8012,263.8187,-92.0176,-38.5324,-11.9756,-101.0145
4,2.5427,-9.7547,-9.4270,-8.9953,-8.6287,-8.0473,-7.2120,0.00,0.0,0.0,...,-104.1133,-11.5732,-92.5401,-9.1177,171.4444,263.9845,-92.5401,-38.6543,-11.9346,-101.7435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,-3.5026,-7.6313,-7.7113,-8.0290,-8.6597,-9.8506,-11.1339,3336.84,614730.0,0.0,...,-99.5915,-11.8236,-87.7679,-5.1304,192.9931,280.7611,-87.7679,-31.6097,-7.8799,-94.8729
8756,-3.2265,-7.8590,-7.9045,-8.1471,-8.6852,-9.7932,-11.0855,3336.84,614730.0,0.0,...,-94.9037,-10.4646,-84.4391,-5.9257,194.2252,278.6643,-84.4391,-31.2295,-8.3767,-90.9528
8757,-3.0027,-8.0373,-8.0760,-8.2650,-8.7243,-9.7447,-11.0400,3336.84,614730.0,0.0,...,-89.8945,-9.8255,-80.0690,-6.2716,197.7397,277.8087,-80.0690,-30.2203,-8.5808,-85.6891
8758,-2.7023,-8.2845,-8.2529,-8.3710,-8.7687,-9.7071,-10.9868,3229.20,594900.0,0.0,...,-91.2361,-9.9526,-81.2835,-7.1116,192.7957,274.0791,-81.2835,-31.6742,-9.4730,-88.2355


In [289]:
X, y = temp_inner.iloc[:,1:],temp_inner.iloc[:,0]

In [290]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [295]:
X_train.shape

(7008, 38)

In [296]:
model = sm.OLS(y_train,X_train).fit()

In [298]:
preds = model.predict(X_test)

In [299]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                delta_t   R-squared (uncentered):                   0.991
Model:                            OLS   Adj. R-squared (uncentered):              0.991
Method:                 Least Squares   F-statistic:                          1.978e+04
Date:                Fri, 29 Nov 2019   Prob (F-statistic):                        0.00
Time:                        14:53:01   Log-Likelihood:                         -104.91
No. Observations:                7008   AIC:                                      283.8
Df Residuals:                    6971   BIC:                                      537.5
Df Model:                          37                                                  
Covariance Type:            nonrobust                                                  
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
temperature_5cm _°C_                         -0.4220      0.003   -143.833      0.000      -0.428      -0.416
temperature_10cm _°C_                        -0.0363      0.005     -7.185      0.000      -0.046      -0.026
temperature_20cm _°C_                         0.0604      0.006      9.902      0.000       0.048       0.072
temperature_30cm _°C_                        -1.6078      0.015   -104.891      0.000      -1.638      -1.578
temperature_50cm _°C_                         1.3862      0.019     71.846      0.000       1.348       1.424
temperature_100cm _°C_                        0.6794      0.008     85.273      0.000       0.664       0.695
rain_accumulation _mm_                    -7.026e-06   2.48e-06     -2.833      0.005   -1.19e-05   -2.16e-06
rain_duration _s_                         -3.369e-08   8.65e-09     -3.896      0.000   -5.06e-08   -1.67e-08
rain_intensity _mm/h_                         0.0044      0.002      2.118      0.034       0.000       0.008
rain_peak_intensity _mm/h_                 2.841e-05      6e-05      0.473      0.636   -8.93e-05       0.000
hail_accumulation _hits/cm^2_                 0.1526      0.241      0.633      0.527      -0.320       0.625
hail_duration _s_                             0.0003      0.000      1.123      0.261      -0.000       0.001
hail_intensity _hits cm^-2 h^-1_             -0.1837      0.864     -0.213      0.832      -1.878       1.510
hail_peak_intensity _hits cm^-2 h^-1_        -0.0851      0.116     -0.736      0.462      -0.312       0.142
wind_direction_minimum _°_                    0.0003   6.37e-05      4.244      0.000       0.000       0.000
wind_direction_average _°_                 -8.68e-05   8.78e-05     -0.988      0.323      -0.000    8.54e-05
wind_direction_maximum _°_                   -0.0001   7.44e-05     -1.713      0.087      -0.000    1.84e-05
wind_speed_minimum _km/h_                    -0.0060      0.001     -4.660      0.000      -0.009      -0.003
wind_speed_average _km/h_                    -0.0026      0.001     -3.956      0.000      -0.004      -0.001
wind_speed_maximum _km/h_                     0.0007      0.000      2.035      0.042    2.41e-05       0.001
temp_air _°C_                                 0.0040      0.002      1.833      0.067      -0.000       0.008
temp_internal _°C_                            0.0119      0.002      5.865      0.000       0.008       0.016
relative_humidity _%RH_                       0.0021      0.000      8.112      0.000       0.002       0.003
air_pressure _hPa_                           -0.0003   5.08e-05     -5.367      0.000      -0.000      -0.000
irradiance_sho